In [ ]:
!pip install streamlit pyngrok joblib scikit-learn


In [ ]:
%%writefile app.py
import streamlit as st
import joblib
import numpy as np

# Load model + vectorizers
model = joblib.load('best_random_forest_model.pkl')
tfidf_desc = joblib.load('tfidf_vectorizer_description.pkl')
tfidf_req = joblib.load('tfidf_vectorizer_requirements.pkl')
tfidf_profile = joblib.load('tfidf_vectorizer_profile.pkl')

st.title("🕵️ Fake Job Posting Detector")

with st.form("form"):
    title = st.text_input("Job Title")
    company_profile = st.text_area("Company Profile")
    description = st.text_area("Job Description")
    requirements = st.text_area("Requirements")
    submitted = st.form_submit_button("Check")

if submitted:
    desc_vec = tfidf_desc.transform([description])
    req_vec = tfidf_req.transform([requirements])
    prof_vec = tfidf_profile.transform([company_profile])
    X_input = np.hstack([desc_vec.toarray(), req_vec.toarray(), prof_vec.toarray()])

    pred = model.predict(X_input)[0]
    prob = model.predict_proba(X_input)[0][1]

    if pred == 1:
        st.error(f"🚨 FAKE job detected! (Confidence: {prob:.2f})")
    else:
        st.success(f"✅ Looks like a REAL job! (Confidence: {1 - prob:.2f})")


Writing app.py


In [ ]:
from pyngrok import conf
conf.get_default().auth_token = "2w8TYkSGZMKYnG7aaQJ0QQZUeFE_4aZ1fGpiQMBPhAe4C23xD"


In [ ]:
from pyngrok import ngrok
!streamlit run app.py &>/content/log.txt &

# Wait 5–10 seconds
import time; time.sleep(10)

# Use bind_tls instead of port
# Setting bind_tls to True or False, or both will create an HTTPv2 tunnel.
# If you want to specify the ports explicitly, use bind_tls like this: bind_tls=f"{port1},{port2}"
url = ngrok.connect(bind_tls=True, proto='http', addr=8501)
print("🔗 Your app is live at:", url)

🔗 Your app is live at: NgrokTunnel: "https://d64b-34-106-77-14.ngrok-free.app" -> "http://localhost:8501"
